In [ ]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('max_columns', None)

# 데이터 준비

In [ ]:
df = pd.read_csv("my_data/naver_finance/2016_12.csv")

In [ ]:
df.head()

## 수익률 구하기 (16.12 ~ 17.12)

In [ ]:
df['rtn'] = df['price2'] / df['price'] - 1

# PER 값에 따라 group number 부여하기

## 값을 기준으로 grouping 하기 (DIFFERENT number of members in each  group)

### boolean selection & loc 사용

- 곧 뒤에서 배울 `cut()` 을 사용하면 아래 방법보다 더 쉽게 가능합니다. 하지만 여기서 진행하는 방식들도 매우 중요하니 반드시 익혀두세요!

In [ ]:
(df['PER(배)'] >= 10).head()

In [ ]:
bound1 = df['PER(배)'] >= 10
bound2 = (5 <= df['PER(배)']) & (df['PER(배)'] < 10)
bound3 = (0 <= df['PER(배)']) & (df['PER(배)'] < 5)
bound4 = df['PER(배)'] < 0

In [ ]:
df.shape

In [ ]:
df[bound1].shape # = df.loc[bound1].shape

In [ ]:
df.loc[bound1, 'PER_Score'] = 1
df.loc[bound2, 'PER_Score'] = 2
df.loc[bound3, 'PER_Score'] = 3
df.loc[bound4, 'PER_Score'] = -1

In [ ]:
df['PER_Score'].head()

In [ ]:
df['PER_Score'].nunique()

In [ ]:
df['PER_Score'].value_counts()

- `PER_Score`가 float number로 나오는 이유?

In [ ]:
df['PER_Score'].hasnans

In [ ]:
df['PER_Score'].isna().sum()

In [ ]:
df['PER(배)'].isna().sum()

In [ ]:
df[df['PER(배)'].isna()]

In [ ]:
df.loc[df['PER_Score'].isna(), "PER_Score"] = 0

# 아래와 같은 방식으로도 가능
# df['PER_Score'] = df['PER_Score'].fillna(0) 
# df.loc[:, 'PER_Score'] = df['PER_Score'].fillna(0)

### boolean series 의 연산 특성 사용

In [ ]:
df.loc[:, "PER_Score1"] = (bound1 * 1)  + (bound2 * 2) + (bound3 * 3) + (bound4 * -1) 

In [ ]:
df['PER_Score1'].head()

In [ ]:
df['PER_Score1'].value_counts()

In [ ]:
df['PER_Score'].value_counts()

### 위의 두 score series는 서로 같을까? 

In [ ]:
df['PER_Score'].equals(df['PER_Score1'])

In [ ]:
df['PER_Score'].dtypes
df['PER_Score1'].dtypes

In [ ]:
df['PER_Score'].astype(int).equals(df['PER_Score1'])

### `cut()`

In [ ]:
per_cuts = pd.cut(
    df['PER(배)'],
    [-np.inf, 0, 5, 10, np.inf], 
)

per_cuts.head()

In [ ]:
per_cuts.iloc[0]

In [ ]:
per_cuts.value_counts()

In [ ]:
per_cuts.isna().sum()

- cut()과 동시에 label 달아주기

In [ ]:
bins = [-np.inf, 10, 20, np.inf]
labels = ['저평가주', '보통주', '고평가주']
per_cuts2 = pd.cut(
    df['PER(배)'], 
    bins=bins, 
    labels=labels
)
per_cuts2.head()

In [ ]:
# df.loc[:, 'PER_score2'] = per_cuts  # or per_cuts2
# df['PER_score2'] = per_cuts         # or per_cuts2

## Group내 데이터 갯수를 기준으로 grouping 하기 (SAME number of members in each  group)

### `qcut()`

In [ ]:
pd.qcut(df['PER(배)'], 3, labels=[1,2,3]).head()

In [ ]:
df.loc[:, 'PER_Score2'] = pd.qcut(df['PER(배)'], 10, labels=range(1, 11))
df.head()

In [ ]:
df['PER_Score2'].value_counts()

In [ ]:
df['PER_Score2'].hasnans

In [ ]:
df['PER_Score2'].isna().sum()

In [ ]:
df['PER_Score2'].dtype

- 'category' type: A string variable consisting of only a few different values

In [ ]:
# DataFrame에서 category dtype인 columns들 추출하기
# df.select_dtypes(include=['category']).columns

In [ ]:
df['PER_Score2'].head()

In [ ]:
df['PER_Score2'].value_counts()

In [ ]:
df = df.dropna(subset=['PER(배)'])

In [ ]:
df['PER_Score2'].isna().sum()

# Split - Apply - Combine 

In [ ]:
df = pd.read_csv("my_data/naver_finance/2016_12.csv")
df.shape

In [ ]:
df = df.dropna()
df.shape

In [ ]:
g_df = df.copy()
g_df.head()

## Group score 생성

In [ ]:
g_df['rtn'] = g_df['price2'] / g_df['price'] - 1

In [ ]:
g_df.loc[:, 'PER_score'] = pd.qcut(g_df['PER(배)'], 10, labels=range(1, 11))
g_df.loc[:, 'PBR_score'] = pd.qcut(g_df['PBR(배)'], 10, labels=range(1, 11))

In [ ]:
g_df.set_index('ticker', inplace=True)

In [ ]:
g_df.head()

In [ ]:
g_df.get_dtype_counts()

## groupby() & aggregation

- `groupby()`
    - 실제로 grouping까지는 하지 않고, grouping이 가능한지 validation만 진행(preparation)
- Aggregation
    - 2가지 요소로 구성
        - aggregating columns
        - aggregating functions
            - e.g. `sum, min, max, mean, count, variacne, std` etc

- 결국, 3가지 요소만 충족시키면 됨!
    - Grouping columns (cateogorial data type)
    - Aggregating columns
    - Aggregating functions

### `groupby` object 살펴보기

In [ ]:
g_df.groupby('PER_score')

In [ ]:
g_df_obj = g_df.groupby(["PBR_score", "PER_score"])
g_df_obj

In [ ]:
type(g_df_obj)

In [ ]:
g_df_obj.ngroups

In [ ]:
g_df['PBR_score'].nunique()
g_df['PER_score'].nunique()

- "ngroups와 (g_df['PBR_score'].nunique() x g_df['PER_score'].nunique())가 차이가 나는 이유"에 대해서 생각해보기

In [ ]:
type(g_df_obj.size())

In [ ]:
g_df_obj.size().head()

In [ ]:
# Multi-level index를 가진 Series indexing하는 법 
g_df_obj.size().loc[1]
g_df_obj.size().loc[(1, 1)]

In [ ]:
# Series -> DataFrame으로 변환
g_df_obj.size().to_frame().head()

In [ ]:
type(g_df_obj.groups)
g_df_obj.groups.keys()
g_df_obj.groups.values ()

In [ ]:
# Retrieve specific group
g_df_obj.get_group((1, 1))

- For loop을 이용해서 grouping된 object 확인해보기 (많이는 안쓰임)

In [ ]:
for name, group in g_df_obj:
    print(name)
    group.head(2)
    break

In [ ]:
# 참고 :groupby()에 대해 head()를 적용하면, 기존이 head()가 작동하는 방식, 즉, 최상위 2개를 가지고 오는게 아니라
# 각 그룹별 최상위 2개를 무작위로 섞어서 하나로 합친 DataFrame을 리턴함
g_df.groupby('PER_score').head(2)

### aggreggation

- 반드시 "aggregating" 기능이 있는 function 을 써야함
    - min, max, mean, median, sum, var, size, nunique, idxmax

In [ ]:
g_df.groupby("PBR_score").agg(
    {
        "rtn": "mean", # =  np.mean
    }
)

In [ ]:
pbr_rtn_df = g_df.groupby("PBR_score").agg({'rtn': 'mean'})
per_rtn_df = g_df.groupby("PER_score").agg({'rtn': 'mean'})

In [ ]:
pbr_rtn_df.head()

In [ ]:
# 다양한 방법으로 진행하기 (같은 결과)
g_df.groupby("PER_score")['rtn'].agg('mean').head()
g_df.groupby("PER_score")['rtn'].agg(np.mean).head()
g_df.groupby("PER_score")['rtn'].mean().head()

In [ ]:
# return type이 다를 수 있음에 주의
g_df.groupby("PER_score")['rtn'].agg("mean").head(2)   # Series로 return
g_df.groupby("PER_score")[['rtn']].agg("mean").head(2)  # DataFrame으로 return

In [ ]:
# 2개 이상의 컬럼에 대해 aggregation
g_df.groupby("PER_score")[['rtn', 'PBR(배)']].agg("mean").head(2)

In [ ]:
# 2개 이상의 aggregation
g_df.groupby("PER_score")[['rtn', 'PBR(배)']].agg(["mean", "std"]).head(2)

In [ ]:
# 2개 이상의 컬럼 & 각각에 대해 다른 aggregation
g_df.groupby("PBR_score").agg(
    {
        'rtn': ['mean', 'std'],
        'PER(배)': ['min']
        
    }
)

- aggregation function이 아닌경우 => `agg()`가 error를 발생시킴


In [ ]:
# sqrt는 aggregation 방식의 연산이 아님!
np.sqrt([1, 2, 3, 4])

In [ ]:
g_df.groupby("PER_score")['rtn'].agg(np.sqrt)

- Visualization(시각화) 맛보기

In [ ]:
%matplotlib inline

In [ ]:
pbr_rtn_df.plot(kind='bar')

In [ ]:
pbr_rtn_df.plot(kind='bar');
per_rtn_df.plot(kind='bar');

### Examples

In [ ]:
g_df1 = g_df.groupby(["PBR_score", "PER_score"])\
            .agg(
                {
                    'rtn': ['mean', 'std', 'min', 'max'],
                    'ROE(%)': [np.mean, 'size', 'nunique', 'idxmax'] 
                 }
            )
g_df1.head()

In [ ]:
a = g_df.groupby(["PBR_score", "PER_score"])['rtn', 'ROE(%)'].agg(['sum', 'mean'])

In [ ]:
# Multi-index라고 해서 쫄 것 없음!
a.loc[1]
a.loc[(1, 3)]
a.loc[[(1, 3), (1, 4 )]]

### 주의: nan은 groupby시 자동으로 filter out 되기 때문에, 미리 전처리 다 하는게 좋음 

In [ ]:
df = pd.DataFrame({
    'a':['소형주', np.nan, '대형주', '대형주'],
    'b':[np.nan, 2,         3,     np.nan],
})
df

In [ ]:
df.groupby(['a'])['b'].mean()

###  `as_index = False` : group cols들이 index가 아니라 하나의 col이 됨 (aggregate하고 reset_index()를 취한 것)

In [ ]:
a = g_df.groupby(["PER_score"]                ).agg({'rtn': ['mean', 'std']}).head(2)
b = g_df.groupby(["PER_score"], as_index=False).agg({'rtn': ['mean', 'std']}).head(2)

In [ ]:
a
b

In [ ]:
a.index
a.columns

In [ ]:
b.index
b.columns

In [ ]:
a['rtn']

In [ ]:
a[('rtn', 'mean')].head()

### Multi-index columns을 하나로 병합하기

In [ ]:
g_df1.head()

In [ ]:
level0 = g_df1.columns.get_level_values(0)
level1 = g_df1.columns.get_level_values(1)

level0
level1

In [ ]:
g_df1.columns = level0 + '_' + level1

In [ ]:
g_df1.head(2)

In [ ]:
g_df1 = g_df1.reset_index()
g_df1.head()

# 실전예제: 시가총액으로 Small and Big 나누기 

In [ ]:
a_df = pd.read_csv("my_data/Small_and_Big.csv", index_col=[0])
a_df.head()

In [ ]:
a_df.tail()

In [ ]:
median_df = a_df.groupby(['date']).agg({'시가총액 (보통)(평균)(원)': 'median'})
median_df.head()

In [ ]:
median_df.columns = ['시가총액_median']
median_df.head()

- 구한 median dataframe을 어떻게 가존의 원본 dataframe과 연결 시킬수있을까?
=> 다음 노트북!